In [2]:
import pandas as pd

# df = pd.read_csv('./source/all_mfw.csv')

In [3]:
df['评论内容'].values[:3]

array(['经历了卧铺大巴后终于在凌晨抵达了青岛，因为是公司组织的活动所以去的地方不是很多。在不觉三个小时后大家一起去了海边欣赏了日出，超姐美，所有的睡意都被赶跑，海水不是特别凉海风都带着热意。我不是很能吃的惯海鲜，在青岛几天顿顿海鲜，不知道是我吃不惯还是烧的不好吃....晚上有篝火晚会，公司几个小年轻拉着小音箱在海边吹着海风跳社会摇（？？？）被一群人围观哈哈哈哈',
       '金沙滩是我国沙质最细、面积最大、风景最美的海滩，号称“亚洲第一滩”。而且这里是免票的，平时大家可以在火车站坐游2（观光2路）直达，可我这赶上了上合峰会，游2停运，只好先坐隧道5路到海韵嘉园然后再换乘L3（新区旅游专线）到达。由于去的不是旺季（5月23日），所以人不多，但景色很漂亮，值得一去。',
       '去金沙滩度假完全是被栈桥上的人潮给吓晕了，在青岛想去个人少、沙滩干净的海滩，金沙滩银沙滩，真是不错的选择。我们觉得简直是海滩包场，虽然天冷不能下水，但孩子只要有沙子，就能玩整整一天，哪怕全身滚成泥猴，我也不会拦着她。'],
      dtype=object)

In [5]:
df[df['标签'] == 'negative']['评论内容'].values[:5]

array(['在黄岛区，沙子以细出名，现在很出名的青岛啤酒节主会场就搬到了金沙滩，本地人很少来的，无非如此。不建议特意来此，除非就住在这附近。',
       '青岛一般喝啤酒可以去有青岛啤酒节玩的没什么好玩的海面也一般', '8月份下饺子一样的人，没法玩！',
       '不知道怎么的，长期关闭了，昨天去的时候出租车师傅就说关闭了',
       '门票10元，不太推荐去，里面没啥意思。没有鲁迅公园好玩，非打卡爱好者还是别来了。在鲁迅公园就挺好。'], dtype=object)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17654 entries, 0 to 17653
Data columns (total 4 columns):
景区名称    17654 non-null object
用户名     17654 non-null object
评论内容    17654 non-null object
标签      17654 non-null object
dtypes: object(4)
memory usage: 551.8+ KB


In [57]:
df.to_csv('./source/all_mfw.csv', index=False, encoding='utf-8')

In [9]:
df = pd.read_csv('./all_mfw.csv')

In [10]:
df['标签'].unique()

array(['positive', 'neutral', 'negative', 'netural', 'negu', 'neg',
       'negea', 'neu', 'neupositive'], dtype=object)

In [3]:
df = pd.read_csv('./source/trainSet.csv')
len(df[df['标签'] == 'positive']) / len(df)

0.8761026139030509

In [4]:
df['标签'].value_counts()

positive    10826
negative      796
neutral       735
Name: 标签, dtype: int64

In [5]:
new_df = df.copy()
df1 = df[df['标签'] == 'negative']
df2 = df[df['标签'] == 'neutral']
for i in range(10):
    new_df = new_df.append(df1)
    new_df = new_df.append(df2)
new_df['标签'].value_counts()

positive    10826
negative     8756
neutral      8085
Name: 标签, dtype: int64

In [6]:
from sklearn.utils import shuffle
new_df = shuffle(new_df)
new_df.to_csv('./source/trainSet{}.csv'.format(len(new_df)), index=False)

In [11]:
import jieba
import re


def data_process_one(text):
    """
    对文本进行预处理：清除无用字符，去除停用词，分词
    :param text: 评论文本，
    :stopwords：停用词
    :return 预处理后的文本
    """
    text = str(text).replace('\n', ' ').strip()
    text = re.sub('\s+', ' ', text)
    text = re.sub('[,，。 /:：]', ',', text)
    text = re.sub(',+', ',', text)
    word_list = list(jieba.cut(text))
    return ' '.join(word_list)

In [12]:
df['cut'] = df['评论内容'].apply(data_process_one)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\kungeek\AppData\Local\Temp\jieba.cache
Loading model cost 0.728 seconds.
Prefix dict has been built successfully.


In [13]:
df['cut'].head()

0    经历 了 卧铺 大巴 后 终于 在 凌晨 抵达 了 青岛 , 因为 是 公司 组织 的 活动...
1    金沙滩 是 我国 沙质 最细 、 面积 最大 、 风景 最美 的 海滩 , 号称 “ 亚洲 ...
2    去 金沙滩 度假 完全 是 被 栈桥 上 的 人潮 给 吓 晕 了 , 在 青岛 想 去 个...
3    ? ? 金沙滩 位于 黄海之滨 , 沙质 细软 , 色泽 如金 , 号称 “ 亚洲 第一 滩...
4    金沙滩 位于 青岛 开发区 东南 , 南 濒 黄海 , 呈 月牙形 东西 伸展 , 全长 3...
Name: cut, dtype: object

In [14]:

with open('./test.txt', 'w', encoding='utf-8') as f:
    for sen in df['cut']:
        f.write(sen)
        f.write('\n')

In [17]:
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence

inp = './test.txt'
model = Word2Vec(LineSentence(inp), size=50, window=5, min_count=5)

In [36]:
model.wv.similar_by_word('经历')

[('心里', 0.9803360104560852),
 ('得出', 0.979602038860321),
 ('遇上', 0.9748328328132629),
 ('脚踝', 0.9739866256713867),
 ('记不清', 0.9738843441009521),
 ('怎样', 0.9734298586845398),
 ('丛林', 0.9728232622146606),
 ('旋转', 0.9724557995796204),
 ('好心', 0.9723654389381409),
 ('立马', 0.9715825915336609)]

In [23]:
for i in model.wv.vocab.items():
    print(i)
    break

('经历', <gensim.models.keyedvectors.Vocab object at 0x00000162E0F67E48>)


In [24]:
from sklearn.model_selection import train_test_split


a, b = train_test_split(df, test_size=0.3)

In [25]:
a.head()

,景区名称,用户名,评论内容,标签,cut
15908,泰安地下龙宫（原泰山大裂谷）,腾玮,景区入口到大裂谷溶洞有一段距离，我们步行前往，其实有点远，带着老人和小孩的还是建议乘坐观光车...,positive,"景区 入口 到 大 裂谷 溶洞 有 一段距离 , 我们 步行 前往 , 其实 有点 远 , ..."
10714,青岛海底世界-青岛水族馆,Cris紀行,中国最早的水族馆如果不看海洋世界门票40学生票20还是非常有性价比的水母馆淡水生物馆海产馆标...,positive,中国 最早 的 水族馆 如果 不看 海洋 世界 门票 40 学生票 20 还是 非常 有 性...
16039,水浒好汉城,妖怪,仿古建筑，你说水浒好汉们如果活过来，看到执政的官员们那么喜欢他们，还愿意造反不？,negative,"仿古 建筑 , 你 说 水浒 好汉 们 如果 活过来 , 看到 执政 的 官员 们 那么 喜..."
7014,唐摩崖石刻,Andrew,泰山景点之一，为历代文人墨客留下的题词,positive,"泰山 景点 之一 , 为 历代 文人墨客 留下 的 题词"
8510,坊茨小镇,四si44,个人觉得一般，挺大一个地方，交通也不是很方便,negative,"个人 觉得 一般 , 挺大 一个 地方 , 交通 也 不是 很 方便"


In [27]:
print(model.wv)

In [37]:
model

In [38]:
model['景区']

D:\software\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':


array([-1.3268019 , -0.09258349, -1.5474279 ,  0.61796343, -1.2365726 ,
        2.2492924 , -1.6366208 ,  0.5004851 , -1.492783  , -0.34838775,
       -0.3294299 , -0.08238316,  1.1463965 , -2.2342906 ,  0.5168381 ,
       -0.3429096 , -0.6035272 , -1.0844334 , -0.18779942,  1.1136084 ,
        0.12346059, -2.243467  ,  1.7600442 , -0.66077673,  0.745564  ,
       -0.34259322,  0.6904377 ,  0.22197996,  1.3596495 , -0.45526338,
        1.565523  , -0.3935501 ,  1.1214525 ,  0.25940618, -1.8716938 ,
       -2.826254  ,  2.9702628 ,  1.3386993 ,  2.2724104 ,  1.5194826 ,
        1.1264942 ,  0.74727285,  0.12674804,  0.94464004,  0.40207508,
        1.4234304 , -0.22212489,  0.65193975, -2.3338816 ,  0.4240599 ],
      dtype=float32)

In [45]:
type(model.wv.index2word)

list

In [46]:
len(model.wv.index2word)

11053

In [47]:
model.wv.index2word[:3]

[',', '的', '是']

In [49]:
model.wv.vectors[1]

array([ 1.7578111 ,  0.00217677,  0.68798524,  0.43423814, -0.2907579 ,
        0.9505683 ,  1.8458284 , -0.5494947 ,  0.20446567, -0.70580107,
       -0.5673537 , -0.49935126, -1.1846464 , -1.1016223 ,  0.11690718,
        0.21718957,  0.0944322 ,  1.3019673 , -1.5068196 ,  0.48519364,
       -0.24316345,  0.45476693, -0.12775567,  1.1276364 ,  0.71076024,
       -0.6999353 ,  1.4405955 , -0.4849769 ,  1.0680168 , -1.9963092 ,
       -0.2815058 ,  0.01951147, -0.36541805,  0.41341236, -0.6503029 ,
        0.26663408, -0.9058468 ,  0.67839515, -1.1354706 ,  0.34066805,
       -0.95137334, -0.01127189, -1.389489  ,  0.21288036,  0.18662569,
        0.8033737 ,  0.3348252 ,  0.6167162 ,  0.12476119,  0.8330245 ],
      dtype=float32)

In [58]:
df['标签'].value_counts()

positive    15493
negative     1102
neutral      1059
Name: 标签, dtype: int64

In [59]:
len(df[df['标签'] == 'positive']) / len(df)

0.8775914806842642